In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/final/'
sys.path.append(project_dir+'pages/')

Mounted at /content/drive


In [2]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/project/final/pages/session_page.py"
"""
Session page for the music recommender app.
Main application interface after login.
"""
import sys
import importlib

# Import modules with reload to avoid circular dependencies
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/app')
import layout
importlib.reload(layout)

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/app')
import navigation
importlib.reload(navigation)
from navigation import navigate_to, show_status, show_loading, get_output

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/app')
import session_state
importlib.reload(session_state)
from session_state import session_state

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/handlers')
import session_handler
importlib.reload(session_handler)
from session_handler import SessionHandler

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/services')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/services')
import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/services')
import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/pages')
import base_page
importlib.reload(base_page)
from base_page import BasePage

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

def render_page():
    """Render the session page."""
    # Create services
    recommendation_service_instance = RecommendationService(
        "/content/drive/MyDrive/Colab Notebooks/project/final/data/songs.csv",
        "/content/drive/MyDrive/Colab Notebooks/project/final/data/listening_history.csv",
        "/content/drive/MyDrive/Colab Notebooks/project/final/data/users.csv"
    )
    emotion_service_instance = EmotionService("/content/drive/MyDrive/Colab Notebooks/project/final/data/emotion_results.csv")
    logging_service_instance = LoggingService("/content/drive/MyDrive/Colab Notebooks/project/final/data/listening_history.csv")

    # Create handlers
    session_handler_instance = SessionHandler(
        recommendation_service_instance,
        emotion_service_instance,
        logging_service_instance
    )

    # Create page
    page = SessionPage(session_handler_instance)
    page.render()

class SessionPage(BasePage):
    """Session page implementation."""

    def __init__(self, session_handler):
        """
        Initialize the session page.

        Args:
            session_handler: Instance of SessionHandler
        """
        super().__init__(title="🎵 Music Recommender")
        self.session_handler = session_handler
        self.mode_selector = None
        self.mode_description = None

    def render(self):
        """Render the session page."""
        # Add title
        self.add_title()

        # User info
        user_info = widgets.HTML(
            f"<p style='text-align:center; margin-bottom:15px;'><b>👤 Signed in as:</b> {session_state.username}</p>"
        )
        self.add_widget(user_info)

        # Mode section
        self.add_subtitle("Select Your Experience Mode:")

        # Mode selector
        self.mode_selector = widgets.ToggleButtons(
            options=[
                ('Private Mode', 'none'),
                ('Log My Activity', 'log'),
                ('Recommend Me Music', 'recommend')
            ],
            value=session_state.mode,
            description='Mode:',
            button_style='info',
            tooltips=[
                'Private Mode: No logging or recommendations',
                'Log Mode: Track your listening activity',
                'Recommend Mode: Get personalized music recommendations'
            ],
            layout=widgets.Layout(width='auto', margin='0 auto')
        )
        self.add_widget(self.mode_selector)

        # Mode description box
        self.mode_description = widgets.HTML()
        self.add_widget(self.mode_description)

        # Update description based on current mode
        self._update_mode_description(session_state.mode)

        # Set up mode change handler
        def on_mode_change(change):
            new_mode = change['new']
            description = self.session_handler.change_mode(new_mode)
            self._update_mode_description(new_mode, description)

        self.mode_selector.observe(on_mode_change, names='value')

        # Actions section
        self.add_subtitle("Actions:")

        # Action buttons
        play_button = widgets.Button(
            description='Play Random Song',
            icon='play',
            button_style='success',
            layout=layout.button_layout
        )

        sad_button = widgets.Button(
            description='I Feel Sad 😢',
            button_style='info',
            layout=layout.button_layout
        )

        happy_button = widgets.Button(
            description='I Feel Happy 😊',
            button_style='info',
            layout=layout.button_layout
        )

        # Button handlers
        def on_play_click(b):
            """Handle play random song button click"""
            show_status("")
            result = self.session_handler.play_random_song()
            status_type = 'info' if "Privacy mode" in result else 'success'
            show_status(result, status_type)

        def on_sad_click(b):
            """Handle I feel sad button click"""
            show_status("")
            message, song_ui = self.session_handler.handle_sad_mood()

            if message:
                show_status(message, 'warning')
            elif song_ui:
                with get_output():
                    clear_output()
                    display(song_ui)

        def on_happy_click(b):
            """Handle I feel happy button click"""
            show_status("")
            result = self.session_handler.handle_happy_mood()
            show_status(result, 'success')

        play_button.on_click(on_play_click)
        sad_button.on_click(on_sad_click)
        happy_button.on_click(on_happy_click)

        # Add action buttons
        action_buttons = widgets.HBox(
            [play_button, sad_button, happy_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )
        self.add_widget(action_buttons)

        # Profile section
        self.add_subtitle("Profile Settings:")

        # Profile buttons
        fav_edit_button = widgets.Button(
            description='Edit Favorites',
            icon='edit',
            layout=layout.button_layout
        )

        exit_button = widgets.Button(
            description='Log Out',
            icon='sign-out',
            button_style='danger',
            layout=layout.button_layout
        )

        # Button handlers
        fav_edit_button.on_click(lambda b: navigate_to('edit_favorites'))
        exit_button.on_click(lambda b: self.session_handler.logout())

        # Add profile buttons
        profile_buttons = widgets.HBox(
            [fav_edit_button, exit_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )
        self.add_widget(profile_buttons)

        # Add output widget
        self.add_widget(get_output())

        # Display the page
        self.display()

    def _update_mode_description(self, mode, description=None):
        """
        Update the mode description.

        Args:
            mode (str): The mode
            description (str): Optional description text
        """
        if description is None:
            # Default descriptions if not provided
            descriptions = {
                'none': "Private Mode: Your activity won't be logged or used for recommendations.",
                'log': "Log Mode: Your listening activity will be tracked to improve recommendations.",
                'recommend': "Recommend Mode: Tell us how you're feeling and get personalized recommendations!"
            }
            description = descriptions.get(mode, "Select a mode to continue.")

        self.mode_description.value = f"""
        <div id="mode-desc-box" style='margin:15px auto; padding:12px; background:#f8f8f8;
        border-radius:8px; width:100%; max-width:500px; text-align:center;'>
            <p id='mode-desc' style='margin:0;'>{description}</p>
        </div>
        """

Overwriting /content/drive/MyDrive/Colab Notebooks/project/final/pages/session_page.py
